<a href="https://colab.research.google.com/github/sudo-logic/Train_GPT_2_on_Discord_Chats/blob/main/Train_GPT_2_on_Discord_Chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Training a GPT-2 Text-Generating Model on Discord Chats

by [Ayush Mishra](https://github.com/sudo-logic)


Retraining a Text Generation Model on Discord Chats using `gpt-2-simple` that wraps existing model fine-tuning and generation scripts for OpenAI's GPT-2.


For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).

##GPU

GPU Hardware Acceleration is needed to make the model training magnitudes faster. 

Make sure you have selected GPU as your Hardware Accelerator in Runtime Settings.

You can verify which GPU is active by running the cell below.

In [1]:
!nvidia-smi

Wed Jan 12 19:18:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    65W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Install and import dependencies

In [2]:
!pip install -q gpt-2-simple 
import gpt_2_simple as gpt2
from datetime import datetime
import requests
from IPython.display import clear_output 

##Addressing Variables
The number of messages and Channel ID to be scraped along with the User/Bot Token for authorization need to be specified.

In [3]:
auth_token = '' #Looks like 'MzgxNDUxMjcxNTg2MTg1MjI3.YWwWOg.CI7-cRZCpGjB8pN4whQxOaOZ4S5'
channel_id = 750068339375603725 #Looks like 750068339375603725
message_count = 30000

##Gathering the Training Data
Scrape the Discord chats using raw GET requests and save them in a data.txt file after proper formatting.

In [4]:
headers = {'authorization': auth_token,}
params = {
  'limit': '100',
}

params['before'] = None

data = []
chat_data = []
for i in range(int(message_count/100)):
  messages = requests.get(f'https://discord.com/api/v9/channels/{channel_id}/messages', headers=headers, params=params).json()
  # print(messages)
  for message in messages:
    data.append(message)
    # print(message)
    chat_data.append(f"{message['author']['username']}: {message['content']}")
  clear_output()
  print(f'{int((len(chat_data)/message_count)*100)}%')

  params['before'] = messages[-1]['id']

print("Scraped Discord Messages Successfully")

content = '\n'.join(chat_data)
open('data.txt','w').write(content)

100%
Scraped Discord Messages Successfully


869560

##Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

---

I have used the "small" model simply because it trains quicker. Larger models will give out more accurate outputs.

In [5]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 265Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.34Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 429Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 48.2Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 361Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 8.44Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 7.25Mit/s]


## Finetuning

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [6]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset='data.txt',
              model_name='124M',
              steps=100,
              restore_from='fresh',
              run_name='run1',
              print_every=1,
              sample_every=20,
              save_every=50
              )


Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


dataset has 376163 tokens
Training...
[1 | 8.20] loss=3.01 avg=3.01
[2 | 12.32] loss=3.04 avg=3.02
[3 | 16.45] loss=2.67 avg=2.90
[4 | 20.58] loss=2.67 avg=2.84
[5 | 24.71] loss=2.79 avg=2.83
[6 | 28.85] loss=3.28 avg=2.91
[7 | 33.00] loss=2.96 avg=2.92
[8 | 37.15] loss=2.81 avg=2.90
[9 | 41.30] loss=2.75 avg=2.89
[10 | 45.47] loss=2.84 avg=2.88
[11 | 49.63] loss=2.67 avg=2.86
[12 | 53.81] loss=2.97 avg=2.87
[13 | 57.99] loss=2.81 avg=2.87
[14 | 62.18] loss=2.95 avg=2.87
[15 | 66.37] loss=2.96 avg=2.88
[16 | 70.56] loss=2.78 avg=2.87
[17 | 74.76] loss=2.86 avg=2.87
[18 | 78.95] loss=2.67 avg=2.86
[19 | 83.16] loss=2.69 avg=2.85
[20 | 87.37] loss=2.67 avg=2.84
======== SAMPLE 1 ========
ip.mp4
MISSISSISSISSIP.mp4
MISSION: Hi!
mihdia: lol
mihdia: so ur gonna say it ur name to the people
mihdia: i am
fizz: I wanna say this :D
mihdia: you wanna say the name im gonna add at the bottom
mihdia: yeah
mihdia: ooo
fizz: D:sadly
xxxy: <:adm1> <a:xpeepoMarsLover:94825586775694844>
fizz: Hi
oPooDan

##Generating Text!

Now the final step, we generate text while tuning the parameters as per our requirement!

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.


In [10]:
gpt2.generate(sess, 
              length=250,
              temperature=0.8,
              # prefix="",
              nsamples=5,
              batch_size=5)

Hello, guys!
Misc 2 was just released: https://tenor.com/view/ugafa-shoot-gif-191555872
okct: just get poggy
Jamal: Asshole.
Misc 2 was just released: https://tenor.com/view/dog-dog-discord-discord-stream-gif-224986979
Classic: Lol
Misc 2 was just released: https://tenor.com/view/innovatives-dog-cat-na-super-dog-gif-18499178
Classic: Yup
Jamal: Oh my.
Misc 2 was just released: https://tenor.com/view/dog-dog-discord-discord-discord-gif-224986979
okct: <:peepoHappy:75837568980963212>
Classic: Hee hee.
Classic: Nothing in this world makes me want to run around the planet doing shit like that
Jamal: You gotta play or die
Classic: nah
Jamal: If you can poof, it's 200
okct:
Dani<3: ass im a stupid guy dani<3: we are both too old for this
𝃔𝒶: yoo
𝃔𝒶: y u are
𝒶𝒶: i cant believe this
Sare: c.us literally like half a dozen of the best guys in the game dont know this person
Sare: c.us tao/comic
𝒶𝒶: no
𝒶𝒶: no
JDW Economy: 
Dani<3: I hope i do this in the future
𝒶𝒶: just tell me
Dani<3: I kinda thi

##Final Thoughts


This was done as a fun little project. The above method is extremely versatile and can be put into use on a multitude of scenarios thanks to how the wrapper package makes it super easy to implement.

##Citations
gpt-2-simple - https://github.com/minimaxir/gpt-2-simple

Max Woolf's Blog - https://minimaxir.com/